In [1]:
import pandas as pd
rides = pd.read_parquet("../data/transformed/validated_rides_2023-06.parquet")
rides.head()

,pickup_datetime,pickup_location_id
0,2023-06-01 00:08:48,140
1,2023-06-01 00:15:04,50
2,2023-06-01 00:48:24,138
3,2023-06-01 00:54:03,100
4,2023-06-01 00:18:44,137


Use the floor function to find the rounded down value of the pickup hour

In [2]:
rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
rides

,pickup_datetime,pickup_location_id,pickup_hour
0,2023-06-01 00:08:48,140,2023-06-01 00:00:00
1,2023-06-01 00:15:04,50,2023-06-01 00:00:00
2,2023-06-01 00:48:24,138,2023-06-01 00:00:00
3,2023-06-01 00:54:03,100,2023-06-01 00:00:00
4,2023-06-01 00:18:44,137,2023-06-01 00:00:00
...,...,...,...
3307229,2023-06-30 23:30:21,42,2023-06-30 23:00:00
3307230,2023-06-30 23:34:22,132,2023-06-30 23:00:00
3307231,2023-06-30 23:45:00,256,2023-06-30 23:00:00
3307232,2023-06-30 23:13:38,91,2023-06-30 23:00:00


Group by the pickup hour and find count of number of pickups in each hour and location. 

In [3]:
agg_rides = rides.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()
agg_rides.rename(columns={0: 'rides'}, inplace=True)
agg_rides


,pickup_hour,pickup_location_id,rides
0,2023-06-01 00:00:00,4,6
1,2023-06-01 00:00:00,7,2
2,2023-06-01 00:00:00,10,1
3,2023-06-01 00:00:00,13,2
4,2023-06-01 00:00:00,24,8
...,...,...,...
73776,2023-06-30 23:00:00,261,19
73777,2023-06-30 23:00:00,262,18
73778,2023-06-30 23:00:00,263,66
73779,2023-06-30 23:00:00,264,42


Keep all zeros for the number of rides in each hour and location. Group by discards this so we need to add them back in. 

In [4]:
from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame:
    
    location_ids = agg_rides['pickup_location_id'].unique()
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H')
    output = pd.DataFrame()
    for location_id in tqdm(location_ids):

        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'rides']]
            
        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        agg_rides_i.set_index('pickup_hour', inplace=True)
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index)
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0)
        
        # add back `location_id` columns
        agg_rides_i['pickup_location_id'] = location_id

        output = pd.concat([output, agg_rides_i])
    
    # move the purchase_day from the index to a dataframe column
    output = output.reset_index().rename(columns={'index': 'pickup_hour'})
    
    return output

In [5]:

agg_rides_all_slots = add_missing_slots(agg_rides)

100%|██████████| 258/258 [00:00<00:00, 434.45it/s]


In [6]:
from typing import Optional, List
import plotly.express as px

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()
     

In [7]:
plot_rides(agg_rides_all_slots, locations=[43])

In [8]:
agg_rides_all_slots.to_parquet('../data/transformed/ts_data_2022_01.parquet')